# Combination of Data Sources

In [1]:
import numpy as np
import pandas as pd
import duckdb 

### GDP Per Capita

GDP Per Capita Dataset: https://data.oecd.org/gdp/gross-domestic-product-gdp.htm

This data set shows the nominal Gross Domestic Product (GDP) per capita of OEPD countries in US dollars from 1960 to 2022. The GDP is the standard measure of the value added created through the production of goods and services in a country during a certain period. The GDP per capita can be found by dividing the total GDP by its population. It also measures the income earned from that population, the total amount spent on final goods and services. This data set was created to be a part of a larger data set called the OECD Economic Outlook created by the OECD Economics Department.

The original data set has 8 columns: Location, Indicator, Subject, Measure, Frequency, Time, Value, Flag Codes). The Location column specifies the country. The Indicator column specifies what is being measured, which is GDP. This column would be helpful when accessing the larger data set, however, in this situation, since we are only accessing the GDP data set, this column is not necessary. The Subject column indicates the topic to which the data values in each row correspond to. In this dataset, since there is only one topic, the only value for this column is “Total GDP”. The Measure column specifies how the data was measured. In this case, the data was measured in US dollars/capita. The Frequency column is similar to the measure column and is used to filter out this specific data set from the larger data set. The Time column indicates what year each data value is from. The time ranges from the years 1960 to 2022. The Value column displays the nominal GDP per capita. The last column, Flag Codes, is used to indicate an error for each row.

The columns: Indicator, Subject, Measure, Frequency, and Flag Codes are not necessary because all the values are the same. They do not provide any significant information.

From the original data set, we removed the Indicator, Subject, Measure, Frequency, and Flag Codes columns. We also renamed the "Location" column as "Country" and the "Time" column as "Year" in order to be consistent with the previous data sets. We also renamed the "Value" column as "GDP in US Dollars". We also need to limit the years to be between the years 1995 to 2020 in order to be consistent with the previous data sets found and limit the amount of missing values in the data.

In [2]:
gdp_df = pd.read_csv('gdp.csv')
query = """
        SELECT 
            LOCATION AS Country,
            TIME AS Year,
            Value AS "GDP per Capita"
        FROM gdp_df
        """
gdp_df = duckdb.sql(query).df()
gdp_df

combined_df = gdp_df

#rounding the GDP values to 2 decimals
combined_df['GDP per Capita'] = combined_df['GDP per Capita'].round(2)

combined_df

,Country,Year,GDP per Capita
0,AUS,1960,2412.63
1,AUS,1961,2383.19
2,AUS,1962,2577.33
3,AUS,1963,2752.62
4,AUS,1964,2902.59
...,...,...,...
1786,CRI,2018,21312.71
1787,CRI,2019,22739.24
1788,CRI,2020,21755.53
1789,CRI,2021,22612.38


### Population Data
Total Population Dataset: https://data.oecd.org/pop/population.htm

This data set shows the total population per millions of people from 1950 to 2022 in OECD countries. The total population count accounts for the following groups: national armed forces stationed abroad, merchant seamen at sea, diplomatic personnel located abroad, civilian aliens resident in the country, and displaced residents in the country. However, this total count excludes foreign armed forces stationed in the country, foreign diplomatic personnel located in the country, and civilian aliens temporarily in the country. 

This data set was created in part of the OECD Labour Force Statistics created by the Committee of Statistics and Statistical Policy to provide reliable annual labor force statistics covering long time series for internal OECD users (in particular, the Economics Department), member country government agencies, and other external users in academic institutions and private enterprise.

The original data set has 8 columns: Location, Indicator, Subject, Measure, Frequency, Time, Value, Flag Codes). The Location specificies which Country. The Indicator column specifies what is being measured, which is population. This column would be helpful when accessing the larger data set (OECD Labour Force Statistics), however, in this situation, since we are only accessing the total population data set, this column is not necessary. The Subject column indicates the topic to which the data values in each row correspond to. In this dataset, since there is only one topic, the only value for this column is “Total Population”. The Measure column just specifies how the data was measured, in this case, the population per millions of people. The Frequency column is similar to the measure column and is used to filter out this specific data set from the larger data set. The Time Column indicates what year each statistic is from. The time ranges from the years 1950 to 2022. The Value column displays the total population for each country and year per million people. The last column, Flag Codes, indicates an error for each row. 

In [3]:
population_df = pd.read_csv('population.csv')
population_df

query = """
        SELECT 
            LOCATION AS Country,
            TIME AS Year,
            Value AS "Population(Million)"
        FROM population_df
        """
population_df = duckdb.sql(query).df()
population_df

,Country,Year,Population(Million)
0,AUS,1950,8.178700
1,AUS,1951,8.421700
2,AUS,1952,8.636500
3,AUS,1953,8.815300
4,AUS,1954,8.986500
...,...,...,...
2842,LTU,2018,2.801543
2843,LTU,2019,2.794137
2844,LTU,2020,2.794885
2845,LTU,2021,2.808380


The two dataframes (combined_df and population_df) are combined below using a left join, so that we will have a cohesive data frame. By using the left join, it allows us to filter through the new data set we are looking to add to our dataset to reduce the number of NAs that may form when there are data for years that is not in our combined_df.

In [4]:
query = """
        SELECT *
        FROM combined_df
        LEFT JOIN population_df
        ON combined_df.Country = population_df.Country
        AND combined_df.Year = population_df.Year;
        """

combined_df = duckdb.sql(query).df()
combined_df 

,Country,Year,GDP per Capita,Country_2,Year_2,Population(Million)
0,AUS,1960,2412.63,AUS,1960,10.275000
1,AUS,1961,2383.19,AUS,1961,10.508200
2,AUS,1962,2577.33,AUS,1962,10.700500
3,AUS,1963,2752.62,AUS,1963,10.906900
4,AUS,1964,2902.59,AUS,1964,11.121600
...,...,...,...,...,...,...
1786,SVN,2022,48361.94,SVN,2022,2.108732
1787,LVA,2012,21297.47,LVA,2012,2.034324
1788,LVA,2014,23810.22,LVA,2014,1.993785
1789,PRT,1977,4136.35,PRT,1977,9.455673


In the table, above there are a lot of missing values. Some countries might not have enrollment rates for either early childhood education or higher education or gdp values and because of this a lot of Column values for Country, and Year are filled with Nan. To fix this, we have to make all the values in the Country and Country_2 column the same, and Year and Year_2 values the same. After doing this, we need to drop the Country_2 and Year_2 columns, because they are redundant and change all the float values in Year to be integers.

In [5]:
#dropping Country_2 and Year_2 columns
combined_df = combined_df.drop(columns=['Country_2', 'Year_2'])

#making the values in the Year column integers
combined_df['Year'] = combined_df['Year'].astype(int)

combined_df

,Country,Year,GDP per Capita,Population(Million)
0,AUS,1960,2412.63,10.275000
1,AUS,1961,2383.19,10.508200
2,AUS,1962,2577.33,10.700500
3,AUS,1963,2752.62,10.906900
4,AUS,1964,2902.59,11.121600
...,...,...,...,...
1786,SVN,2022,48361.94,2.108732
1787,LVA,2012,21297.47,2.034324
1788,LVA,2014,23810.22,1.993785
1789,PRT,1977,4136.35,9.455673


### Education Spending on Higher Education 

Education Spending Data: https://data.oecd.org/eduresource/education-spending.htm#indicator-chart

This data set shows the average amount of spending on tertiary education that covers expenditure on schools, universities and other public and private educational institutions in 38 OECD countries. Spending includes instruction and ancillary services for students and families provided through educational institutions. Education spending is shown in USD per student.

This data set was created in part of a larger data set, Education at a Glance in order to provide descriptive information on the output of educational institutions; the impact of learning across countries; access, participation and progression in education; the financial resources invested in education; and teachers, the learning environment and the organisation of schools. This data was created with the intention to facilitate the dissemination and use of comparative education statistics that will contribute to policies that results in better functioning and more effective education systems.

The original data set has 8 columns: Location, Indicator, Subject, Measure, Frequency, Time, Value, Flag Codes). The Location specificies which Country. The Indicator column specifies what is being measured, which is education expenses. This column would be helpful when accessing the larger data set (Education Spending), however, in this situation, since we are only accessing the total population data set, this column is not necessary. The Subject column indicates the topic to which the data values in each row correspond to. In this dataset, since there is only one topic, the only value for this column is “Tertiary”, meaning that is it not necessary to us. The Measure column just specifies how the data was measured, in this case, USD per student. The Frequency column is similar to the measure column and is used to filter out this specific data set from the larger data set. The Time Column indicates what year each statistic is from. The time ranges from the years 1995 to 2020. The Value column displays the average amount of money spent on tertiary education in US dollars per student. The last column, Flag Codes, is used to indicate an error for each row.

The columns: Indicator, Subject, Measure, Frequency, and Flag Codes are not necessary because all the values are the same. They do not provide any significant information. From the original data set, we removed the Indicator, Subject, Measure, Frequency, and Flag Codes columns. We also renamed the "Location" column as "Country" and the "Time" column as "Year" in order to be consistent with the enrollment_rates_df. We also renamed the "Value" column as "Average Spending on Higher Education (USD/student)" in order to specify the value of it, which is needed when we combine all the data sets.

In [6]:
average_spending_df = pd.read_csv('education_spending.csv')

query = """
        SELECT 
            LOCATION AS Country,
            TIME AS Year,
            Value AS "Average Spending on Higher Education (USD/student)"
        FROM average_spending_df
        ORDER BY Year
        """
average_spending_df = duckdb.sql(query).df()
average_spending_df

,Country,Year,Average Spending on Higher Education (USD/student)
0,CZE,1995,7846.0600
1,HUN,1995,6369.8950
2,CHL,1995,4452.8050
3,FIN,1995,9831.3360
4,USA,1995,15696.5100
...,...,...,...
490,TUR,2020,9287.7930
491,COL,2020,4980.6108
492,LVA,2020,13043.3500
493,CRI,2020,15424.3000


In [7]:
query = """
        SELECT *
        FROM combined_df
        LEFT JOIN average_spending_df
        ON combined_df.Country = average_spending_df.Country
        AND combined_df.Year = average_spending_df.Year;
        """

combined_df = duckdb.sql(query).df()

#dropping Country_2 and Year_2 columns
combined_df = combined_df.drop(columns=['Country_2', 'Year_2'])

#making the values in the Year column integers
combined_df['Year'] = combined_df['Year'].astype(int)

combined_df

,Country,Year,GDP per Capita,Population(Million),Average Spending on Higher Education (USD/student)
0,AUS,2000,28312.86,19.028802,12500.200
1,AUS,2005,35659.13,20.176844,14171.660
2,AUS,2008,40130.34,21.249199,15768.220
3,AUS,2009,41672.92,21.691653,16589.220
4,AUS,2010,42816.43,22.031750,16300.980
...,...,...,...,...,...
1786,USA,2008,48498.45,304.093966,26949.290
1787,EST,2019,39068.37,1.326855,17243.690
1788,USA,2011,49951.91,311.583481,26202.940
1789,SVK,1995,8695.70,5.363676,4851.865


### Expenditures on Education as a Percent of the GDP

This data set shows government expenditures on education as a percentage of total GDP for all the countries in the world from 1980 - 2022. Government expenditure on education (% of GDP) is calculated by dividing the total government expenditure for all levels of education (early, primary, secondary, tertiary, etc.) by the GDP and multiplying by 100. All the data is based on World Bank median estimates. The data on education was collected by the UNESCO Institute for Statistics from official responses to its annual education survey. All the data are mapped to the International Standard Classification of Education (ISCED) to ensure the comparability of education programs at the international level. The current version was formally adopted by UNESCO Member States in 2011. GDP data come from the World Bank.

In the original data set, each row represents an OECD country, each column reflects the school year for which the data was presented, and each value in the data frame represents the percentage of GDP the government spent on education. In some countries, the school year spans across two calendar years, so in these cases, the reference year refers to the year in which the school year ended. There was also no header for the Country column. In order to be consistent with the format of the previous data sets, we melted the data set, added the header for the Country column, and mapped all the country names to their corresponding country codes. We also need to limit Years to be between 1995 and 2020 to be consistent with the previous data sets and limit the amount of missing values.

Some government expenditure percentages might appear lower in some countries where the private sector (households) have a larger share in the total funding for education.

In [8]:
government_expenditure_df = pd.read_csv('total-government-expenditure-on-education-gdp.csv')

query = """
        SELECT 
            Code AS Country,
            Year,
            "Historical and more recent expenditure estimates" AS "Government Expenditure On Education (%)"
        FROM government_expenditure_df
        """
    
government_expenditure_df = duckdb.sql(query).df()
government_expenditure_df

,Country,Year,Government Expenditure On Education (%)
0,AFG,1971,1.16036
1,AFG,1972,1.11718
2,AFG,1973,1.42788
3,AFG,1975,1.30332
4,AFG,1979,1.73981
...,...,...,...
5174,ZWE,2014,6.13835
5175,ZWE,2015,5.81279
5176,ZWE,2016,5.47262
5177,ZWE,2017,5.81878


In [9]:
query = """
        SELECT *
        FROM combined_df
        LEFT JOIN government_expenditure_df
        ON combined_df.Country = government_expenditure_df.Country
        AND combined_df.Year = government_expenditure_df.Year;
        """

combined_df = duckdb.sql(query).df()
combined_df.head()

#dropping Country_2 and Year_2 columns
combined_df = combined_df.drop(columns=['Country_2', 'Year_2'])

combined_df 

,Country,Year,GDP per Capita,Population(Million),Average Spending on Higher Education (USD/student),Government Expenditure On Education (%)
0,AUS,1960,2412.63,10.275000,NaN,1.40000
1,AUS,1978,8553.93,14.359255,NaN,5.99879
2,AUS,1979,9456.68,14.515729,NaN,5.88711
3,AUS,1980,10478.42,14.695356,NaN,5.64446
4,AUS,1982,11988.84,15.184247,NaN,5.47011
...,...,...,...,...,...,...
1786,USA,1973,6725.41,211.908788,NaN,NaN
1787,USA,1982,14399.35,231.664458,NaN,NaN
1788,USA,1990,23835.32,249.622814,NaN,NaN
1789,CHL,2021,28070.41,19.678363,NaN,NaN


### Expenditures on Tertiary Education as a Percent of Total Government Expenditure

Data source: https://databank.worldbank.org/indicator/SE.XPD.TOTL.GD.ZS?id=c755d342&report_name=EdStats_Indicators_Report&populartype=series#

This data show government expenditures on tertiary education as a percentage of total goverment expenditure from 1960 to 2019. Similar to the previous factor of expenditures on education as a whole, this can help us analyze the impact of government support to people completing a higher education on percentage of population with a tertiary education. In addition, the percentage of total expenditures spent on tertiary education can also help quantify how important obtaining pursuing a higher education is to that specific country. If a country's people places an emphasis on higher education, we expect this to reflect in a higher percentage of total government expenditure allocated for higher education.   

The orginal data set has all the countries and all the years since 1960. However, many of the entries are missing values. When downloaded, the countries column did not have a header; to ease manipulation of this data set, I renamed the first column header to "Country".

Since the years are the headers of the columns, which can make it difficult to match enteries during queries to join two dataframes, the years are melted into one single column named "Year".

In addition, since the other dataframes record countries as the capital abbreviations of each, to keep the country names consistant in preparation for the final merge into a dataframe, we mapped the country names to the abbreviations and modified the "Country" column.

In [10]:
gdpPerTertEdu_df = pd.read_csv("TertiaryGovExp%GDP.csv")
gdpPerTertEdu_df = gdpPerTertEdu_df.rename(columns ={" ": "Country"})

# Removing empty column and OECD Member Data
gdpPerTertEdu_df = gdpPerTertEdu_df.drop(columns=['Unnamed: 11'])
#gdpPerTertEdu_df = gdpPerTertEdu_df.drop([27])

# Melting years into a single column
year_names = gdpPerTertEdu_df.columns[1:]
gdpPerTertEdu_df = gdpPerTertEdu_df.melt(id_vars = ["Country"],
        var_name = "Year",
        value_vars = year_names,
        value_name = "Government Spending Teritary (% Gov Spending)")

combined_df["Government Spending Teritary (% Gov Spending)"] = pd.to_numeric(combined_df["Government Spending Teritary (% Gov Spending)"], errors="coerce")

KeyError: 'Government Spending Teritary (% Gov Spending)'

When we were mapping the countries, we decided not to map the average of the OECD members because the different economic and political structure may skew the average data. Also, may of our other data did not contain this kind of value, therefore, to stay consistent, we decided to omit this.

In [ ]:
country_map = {"Australia":"AUS","Austria":"AUT","Belgium":"BEL",
            "Canada":"CAN", "Chile":"CHL", "Colombia":"COL",
            "Costa Rica":"CRI","Czechia":"CZE", 
            "Denmark":"DNK", "Estonia": "EST", 
            "Finland":"FIN", "France":"FRA", "Germany":"DEU", 
            "Greece":"GRC", "Hungary":"HUN", "Iceland":"ISL", 
            "Ireland":"IRL","Israel":"ISR", "Italy":"ITA", 
            "Japan":"JPN", "Korea, Rep.":"KOR", "Korea":"KOR", 
            "Latvia":"LVA", "Lithuania":"LTU", "Luxembourg":"LUX", 
            "Mexico":"MEX", "Netherlands":"NLD", "New Zealand":"NZL",
            "Norway":"NOR", "Poland":"POL", "Portugal":"PRT", 
            "Slovak Republic":"SVK","Slovenia":"SVN","Spain":"ESP",
            "Sweden":"SWE","Switzerland":"CHE", "Turkiye":"TUR",
            "United Kingdom":"GBR","United States":"USA"}

gdpPerTertEdu_df["Country"] = gdpPerTertEdu_df["Country"].map(country_map)
gdpPerTertEdu_df

In [ ]:
query = """
        SELECT *
        FROM combined_df
        LEFT JOIN gdpPerTertEdu_df
        ON combined_df.Country = gdpPerTertEdu_df.Country
        AND combined_df.Year = gdpPerTertEdu_df.Year
        """

combined_df = duckdb.sql(query).df()

#dropping Country_2 and Year_2 columns
combined_df = combined_df.drop(columns=['Country_2', 'Year_2'])

combined_df

### Household Income per Capita

Data source: https://data.oecd.org/hha/household-disposable-income.htm#indicator-chart

This data set shows the gross household disposable income per capita in 38 OECD countries. Household disposable income is available to households such as wages and salaries, income from self-employment and unincorporated enterprises, income from pensions and other social benefits, and income from financial investments. Gross means that depreciation costs are not subtracted and it also includes social transfers, such as health or education provided for free or at reduced prices by governments and not-for-profit organizations.

This data set was created in part of a larger data set, National Accounts of OECD Countries created by the Committee of Statistics and Statistical Policy in order to on an internationally comparable basis, a timely update of annual national accounts data to internal and external users for analytical purposes. It also provides a forum of international exchange on national accounts standards, in order to improve the relevance of SNA and enhance international comparability.

The original data set has 8 columns: Location, Indicator, Subject, Measure, Frequency, Time, Value, Flag Codes). The Location specificies which Country. The Indicator column specifies what is being measured, which is household disposable income. This column would be helpful when accessing the larger data set (National Accounts of OECD Countries), however, in this situation, since we are only accessing the total population data set, this column is not necessary. The Subject column indicates the topic to which the data values in each row correspond to. In this dataset, since there is only one topic, the only value for this column is “Gross Adjusted”. The Measure column just specifies how the data was measured, in this case, gross household income per capita. The Frequency column is similar to the measure column and is used to filter out this specific data set from the larger data set. The Time Column indicates what year each statistics is from. The time ranges from the years 1950 to 2022. The Value column displays the total population for each country and year per millions of people. The last column, Flag Codes, is used to indicate an error for each row.

The columns: Indicator, Subject, Measure, Frequency, and Flag Codes are not necessary because all the values are the same. They do not provide any significant information.

From the original data set, we removed the Indicator, Subject, Measure, Frequency, and Flag Codes columns. We also renamed the "Location" column as "Country" and the "Time" column as "Year" in order to be consistent with the other data frames. We also renamed the "Value" column as "Household Income per Capita" in order to specify the value of it, which is needed when we combine all the data sets.

We also need to limit Years to be in between 1995 to 2020 in order to be consistent with the previous data sets and limit the amount of missing values.

In [ ]:
household_income_df = pd.read_csv('household_income.csv')

query = """
        SELECT 
            LOCATION AS Country,
            TIME AS Year,
            Value AS "Household Income per Capita"
        FROM household_income_df;
        """
household_income_df = duckdb.sql(query).df()
household_income_df

The two dataframes (combined_df and household_income_df) are combined below using a left join, so that we will have a cohesive data frame showing enrollment rates, gdp, and total population, public spending as a percentage of GDP, private spending as a percentage of GDP, and household income per capita of each country from 2013-2020.

In [ ]:
query = """
        SELECT *
        FROM combined_df
        LEFT JOIN household_income_df
        ON combined_df.Country = household_income_df.Country
        AND combined_df.Year = household_income_df.Year;
        """

combined_df = duckdb.sql(query).df()
combined_df 


#dropping Country_2 and Year_2 columns
combined_df = combined_df.drop(columns=['Country_2', 'Year_2'])

#making the values in the Year column integers
combined_df['Year'] = combined_df['Year'].astype(int)

combined_df

### Number of Universities

Data source: https://www.webometrics.info/en/distribution_by_country

While looking for the number of universities in each country, we came across this data set in Statistica: https://www.statista.com/statistics/918403/number-of-universities-worldwide-by-country/, However, we couldn’t figure out a way of downloading the entire csv, instead only the chart shows up, not the original data. This is when we realized that the people who made this chart got their information directly from this other website: https://www.webometrics.info/en/distribution_by_country

The "Webometrics Ranking of World Universities" is an initiative of the Cybermetrics Lab, a research group belonging to the Consejo Superior de Investigaciones Científicas (CSIC), the largest public research body in Spain. They created this dataset because they wanted to create a representative sample of all the universities in all countries and their rankings in the country for useful economic-related analysis. This organization collaborated with other institutions of the Spanish R&D system, and with social, economic, national or foreign agents. The original data set has 7 columns: Country, Top 100, 101-500, 501, 1000, 1001-10000, and Total. The Country column lists all the countries in the world and each other category lists how many universities in each country fall in those rankings.

For our use, we used the Country and the Total column because we didn’t need the rankings of the universities, but how many universities are in each country. However, because, we weren’t able to directly download the full data set, had to create our own csv in Excel by manually entering the data for the country codes of OECD and their corresponding number of universities. When manually entering the data, we also made sure to format it in a way that wecould directly use the data set without cleaning it.

In [ ]:
num_universities_df = pd.read_csv('number_of_universities.csv')
num_universities_df

combined_df = pd.merge(combined_df, num_universities_df, 
                       on='Country', how='left')
combined_df

### Education Enrollment Data

https://ourworldindata.org/global-education

This data set shows the gross rate at which people enter primary, secondary, and tertiary education. It was created to show how school enrollment around the world increased dramatically in the last century. This data set was created by Our World Data, which is a project of the Global Change Data Lab, which is a registered charity in England and Wales.

This data set has 5 columns: Country, Year, and School enrollment, primary (% gross), School enrollment, secondary (% gross), and School enrollment, tertiary (% gross).

The Country column lists all the country codes for all the countries that have available data. For this assignment, we are limiting the list of countries to OECD countries, 38 total countries. The Year column lists all the years over which this data was collected from 1970 to 2020. However, in order to reduce the amount of missing data in our final data set, we are going to limit the range of years from 1995 to 2020.

The rate of primary school enrollment is measured through administrative data and is defined as the number of children enrolled in primary school who belong to a certain age group that corresponds to primary schooling divided by the total population of that age group. Some percentages are greater than 100, because children may enter education late or repeat a year. The rate of enrollment in secondary education and tertiary education is also measured in a similar manner as the rate of enrollment in primary education.

In [ ]:
enrollment_rates_df = pd.read_csv('enrollment_completion_rates.csv')

query = """
        SELECT
            Code AS Country,
            Year,
            "School enrollment, primary (% gross)" AS "Primary Enrollment rate (% gross)",
            "School enrollment, secondary (% gross)" AS "Secondary Enrollment rate (% gross)",
            "School enrollment, tertiary (% gross)" AS "Tertiary Enrollment rate (% gross)"
        FROM enrollment_rates_df
        """

enrollment_rates_df = duckdb.sql(query).df()
enrollment_rates_df.head()

Using left join and mapping the year and country, we can filter out all countries that are not OECD countries, since all OECD countries would already be in the combined_df dataframe.

In [ ]:
query = """
        SELECT *
        FROM combined_df
        LEFT JOIN enrollment_rates_df
        ON combined_df.Country = enrollment_rates_df.Country
        AND combined_df.Year = enrollment_rates_df.Year
        ORDER BY combined_df.Country, combined_df.Year;
        """

combined_df = duckdb.sql(query).df()

#dropping Country_2 and Year_2 columns
combined_df = combined_df.drop(columns=['Country_2', 'Year_2'])

#making the values in the Year column integers
combined_df['Year'] = combined_df['Year'].astype(int)
combined_df

### Population who have completed Tertiary Education

https://data.oecd.org/eduatt/population-with-tertiary-education.htm

This data set shows the population with tertiary education, those who have completed the highest level of education by age group from 1981 to 2022. Tertiary education includes bachelor's and higher levels of education. The measure is the percentage of the same-age population.

This data set was created in part of a larger data set, Education at a Glance in order to provide descriptive information on the output of educational institutions; the impact of learning across countries; access, participation, and progression in education; the financial resources invested in education; and teachers, the learning environment and the organization of schools. This data was created with the intention to facilitate the dissemination and use of comparative education statistics that will contribute to policies that result in better functioning and more effective education systems.

The original data set has 8 columns: Location, Indicator, Subject, Measure, Frequency, Time, Value, Flag Codes). The Location specificies which Country. The Indicator column specifies what is being measured, which is tertiary education. This column would be helpful when accessing the larger data set (Education Spending), however, in this situation, since we are only accessing the total population data set, this column is not necessary. The Subject column indicates the topic to which the data values in each row correspond to. In this dataset, since there are multiple topics since all the data is split by different age groups (25-34 year-olds and 55-64 year-olds). The Measure column just specifies how the data was measured, in this case, a ratio of full-time students to teachers. The Frequency column is similar to the measure column and is used to filter out this specific data set from the larger data set. The Time Column indicates what year each statistic is from. The time ranges from the years 2013 to 2020. The Value column displays the ratio of students enrolled to teachers. The last column, Flag Codes, is used to indicate an error for each row.

The columns: Indicator, Measure, Frequency, and Flag Codes are not necessary because all the values are the same. They do not provide any significant information. From the original data set, we removed the Indicator, Measure, Frequency, and Flag Codes columns. We also renamed the "Location" column as "Country" and the "Time" column as "Year" in order to be consistent with the enrollment_rates_df. We also renamed the "Value" column as "Population with Tertiary Education(%)" in order to specify the value of it, which is needed when we combine all the data sets.

We need to limit the Subject column so the data only includes data from the age group 25-34 year olds so we limit the amount of people we are double counting.

In [ ]:
completed_tertiary_edu_df = pd.read_csv('completed_tertiary_edu.csv')

query = """
        SELECT 
            LOCATION AS Country,
            TIME AS Year,
            Value AS "Population with Tertiary Education (%)",
            SUBJECT
        FROM completed_tertiary_edu_df
        WHERE SUBJECT = '25_34';
        """
completed_tertiary_edu_df = duckdb.sql(query).df()

#dropping the column "subject" since we dont need it anymore
query = """
        SELECT Country, Year, "Population with Tertiary Education (%)"
        FROM completed_tertiary_edu_df;
        """
completed_tertiary_edu_df = duckdb.sql(query).df()
completed_tertiary_edu_df

In [ ]:
query = """
        SELECT *
        FROM combined_df
        LEFT JOIN completed_tertiary_edu_df
        ON combined_df.Country = completed_tertiary_edu_df.Country
        AND combined_df.Year = completed_tertiary_edu_df.Year
        ORDER BY combined_df.Country,combined_df.Year;
        """

combined_df = duckdb.sql(query).df()
combined_df.head()

#dropping Country_2 and Year_2 columns
combined_df = combined_df.drop(columns=['Country_2', 'Year_2'])

#making the values in the Year column integers
combined_df['Year'] = combined_df['Year'].astype(int)

combined_df.head()

When we were combining dataframes and adding different x variables that we want, we did left joins using the combined_df. This acheives our goal of only selecting OECD countries because the first dataframe, GDP, only contains information about OECD countries. However, we did not implement any limitations on year because we want to see the completeness of the final dataframe before we limit the year to prevent losing useful data. Looking at the data description of all our individual dataframes, we decided it would be best to limit the year range of our research to 1995 to 2019.

In [ ]:
# Updates combined_df after filtering the year
query = """
        SELECT *
        FROM combined_df
        WHERE (Year >= 1995) AND (Year < 2020)
        """

combined_df = duckdb.sql(query).df()

combined_df.head()

In [ ]:
combined_df.to_csv('combined_data.csv', index=False)